# *By forking this notebook you will be able to start building your machine learning model, without need to merge tables or clean*

The last version was deleted by mistake. Sorry for that. However, this one contains all what that contained and much more :)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
weather = pd.read_csv('../input/weather-dataset-in-antwerp-belgium/weather_in_Antwerp.csv', ';')
power_info = pd.read_csv('../input/solarpanelspower/PV_Elec_Gas2.csv')
display(weather.head())
power_info.head()

# **Let's clean solar power table first:**

In [ ]:
power_info = power_info[['Unnamed: 0','cum_power']]
power_info = power_info.rename(columns= {'Unnamed: 0': 'date'})
power_info.info()

We have date column as "object" dtype, to transform it to pandas' datetime dtype we do: 

In [ ]:
power_info.date = pd.to_datetime(power_info.date)
power_info.set_index(['date'], inplace=True)       #change the index
power_info.head()

In [ ]:
power_info = power_info.shift(periods=-1, freq='D', axis=0)   #Correcting the measure error
                                                        # (mentioned in Frank's data description)
    
#Calculating daily power, because we have the cumulative one
temp = power_info.shift(periods=1, freq='D', axis=0)
power_info['day_power'] = power_info.loc[:, 'cum_power'] - temp.loc[:, 'cum_power']
power_info.drop(['cum_power'], axis=1, inplace=True)
power_info.day_power.iloc[0] = 5
power_info.head()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

sns.set()
power_index= power_info.reset_index()
power_index.plot(kind='line', x='date', y='day_power', figsize=(15,5))

plt.title('Daily Power Produced By Solar Panels')
plt.ylabel('Daily Power')
plt.show()

# **To clean weather table now:**

In [ ]:
def clear_wind(obj):
    if isinstance(obj, str):
        if obj == 'No wind':
            obj = 0
        else:
            obj = obj.replace(' km/h', '')
    return obj
def trans_from_objects(weather):
    weather.drop(['Unnamed: 0', 'visibility'], axis =1, inplace=True)
    weather.barometer = weather.barometer.apply(lambda x: x.replace(' mbar', '') 
                                    if isinstance(x, str) else x).astype(float)
    
    weather.humidity = weather.humidity.apply(lambda x: x.replace('%', '') 
                                    if isinstance(x, str) else x).astype(float)
    weather.temp = weather.temp.apply(lambda x: x.replace('°C', '') 
                                    if isinstance(x, str) else x).astype(float)
    weather.wind = weather.wind.apply(clear_wind).astype(float)
    
    return weather

#transfer dataframe from objects dtype to numbers
weather_tran = trans_from_objects(weather)
weather_tran.head()

In [ ]:
weather_tran.info()

In [ ]:
#Form the date column 
def create_date(weather):    
    weather['date'] = weather.apply(lambda row:
                                    f'{row.year}-{row.month}-{row.day} {row.clock}', axis=1)
    weather.date = pd.to_datetime(weather.date)
    return weather.drop(['clock', 'year', 'month', 'day'], axis = 1)

weather_pretty = create_date(weather_tran)
weather_pretty.head()

In [ ]:
#to take the average of each day, so we have daily weather. Because we have the daily cum_power not hourly
import datetime 
def take_average_weather(weather):
    average_weather = pd.DataFrame(columns = ['temp', 'weather', 'wind', 'humidity', 'barometer',
                                              'date'])

    temp, wind, humidity, barometer, counter= [0]*5
    for i in range(len(weather)):
        if (weather.loc[i, 'date'].time() ==datetime.time(0, 20)) and (i!=0):
            average_weather = average_weather.append({
                'temp':temp/counter,
                'wind':wind/counter,
                'humidity':humidity/counter,
                'barometer':barometer/counter,
                'date':pd.to_datetime(weather.loc[i-1, 'date'].date()),
                'weather':weath
            }, ignore_index=True)
            temp, wind, humidity, barometer, counter= [0]*5
            
        #Here we'll take the weather status in the most powerful hour (15:20), because you can't take averge 
                                                                                                    #here.
        if (weather.loc[i, 'date'].time()==datetime.time(15,20)):
            weath = weather.loc[i, 'weather']
        counter += 1
        temp += weather.loc[i, 'temp']
        wind += weather.loc[i, 'wind']
        humidity += weather.loc[i, 'humidity']
        barometer += weather.loc[i, 'barometer']
        
    return average_weather
average_weather = take_average_weather(weather_pretty)

In [ ]:
def merge_weatherANDpower():
    dataset = average_weather.merge(power_info, on=['date'])
    return dataset.set_index('date')
final_dataset = merge_weatherANDpower()
final_dataset.head()

In [ ]:
import seaborn as sns
weather_counts = final_dataset.weather.value_counts()
plt.figure(figsize=(16,5))
sns.barplot(weather_counts.index, weather_counts.values, alpha=0.8)
plt.xticks(rotation=90)
plt.title('Weather Status')
plt.xlabel('Status')
plt.ylabel('Number Of Repetition')
plt.show() # WHAT THE HECK! Let's reduce this amount of redundant information

In [ ]:
#I need this, so I can deal with "loc"
final_dataset = final_dataset.reset_index()

#Delete all first parts of two-part status, and highligh only the necessary categories. 
#why the first part? Because we don't care about the raining or snowing weather, we care more about 
#status of clouds
for i in range(len(final_dataset)):
    weather_list = final_dataset.loc[i, 'weather'].split('.')
    if len(weather_list) > 2:
        final_dataset.loc[i,'weather'] = weather_list[1].strip()
    elif len(weather_list) ==2:
        final_dataset.loc[i, 'weather'] = weather_list[0].strip()
final_dataset.head()

#get the index back to "date"
final_dataset.set_index('date', inplace=True)

In [ ]:
from matplotlib import pyplot as plt
final_dataset.weather.value_counts()
weather_counts = final_dataset.weather.value_counts()

plt.figure(figsize=(12,6))
sns.barplot(weather_counts.index, weather_counts.values, alpha=0.8)
plt.xticks(rotation=33)
plt.title('Weather Status')
plt.xlabel('Status')
plt.ylabel('Number Of Repetition')
plt.show()

In [ ]:
final_dataset.info()

In [ ]:
final_dataset.hist(figsize=(16,12))
plt.show()

# Now, Go and build you ML model

Use final_dataset dataframe, remove the nan value, and that's it! Build your model